In [ ]:
!pip install unsloth
!pip uninstall unsloth -y && pip install --upgrade --no-cache-dir --no-deps git+https://github.com/unslothai/unsloth.git

In [ ]:
from unsloth import FastLanguageModel
import torch

max_seq_length = 2048
dtype = None
load_in_4bit = True

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name="unsloth/Meta-Llama-3.1-8B",
    max_seq_length=max_seq_length,
    dtype=dtype,
    load_in_4bit=load_in_4bit
)

In [ ]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16,
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0,
    bias = "none",
    use_gradient_checkpointing = "unsloth",
    random_state = 3407,
    use_rslora = False,
    loftq_config = None,
)

In [ ]:
alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
Answer the legal question and explain the reasoning behind your answer

### Input:
{}

### Response:
{}

Explanation: {}
"""

In [ ]:
EOS_TOKEN = tokenizer.eos_token
def formatting_prompts_func(examples):
  questions = examples["Question"]
  answers = examples["Response"]
  reasonings = examples["Reasoning"]

  texts = []
  for question, answer, reasoning in zip(questions, answers, reasonings):
    text = alpaca_prompt.format(question, answer, reasoning) + EOS_TOKEN
    texts.append(text)
  return {"text": texts}

In [ ]:
# Example Dataset
from datasets import load_dataset
dataset = load_dataset("relai-ai/legal-scenarios-SCOTUS-2024-decisions", split="train")
dataset = dataset.map(formatting_prompts_func, batched=True)

In [ ]:
# To use Formatly dataset you will have to upload the csv data on Google Collab Runtime
dataset = dataset.map(formatting_prompts_func, batched=True)
EOS_TOKEN = tokenizer.eos_token
def formatting_prompts_func(examples):
  questions = examples["question"]
  answers = examples["answer"]
  reasonings = examples["explanation"]

  texts = []
  for question, answer, reasoning in zip(questions, answers, reasonings):
    text = alpaca_prompt.format(question, answer, reasoning) + EOS_TOKEN
    texts.append(text)
  return {"text": texts}

In [ ]:
questions = dataset['Question']
questions[0]

In [ ]:
dataset['text'][0]

In [ ]:
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False, # Can make training 5x faster for short sequences.
    args = TrainingArguments(
        per_device_train_batch_size = 1,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        # num_train_epochs = 1, # Set this for 1 full training run.
        max_steps = 60,
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
        report_to = "none",
    ),
)

In [ ]:
try:
  trainer_stats = trainer.train()
  print("Training completed successfully!")
  print(f"Training stats: {trainer_stats}")
except RuntimeError as e:
  print(f"Runtime error: {e}")

In [ ]:
model.save_pretrained_gguf("model", tokenizer, quantization_method = "f16")

In [ ]:
model.push_to_hub_gguf("YOUR HUGGINGFACE REPO", tokenizer, quantization_method = "f16", token = "YOUR HUGGINGFACE TOKEN")